In [1]:
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import os
import pinecone
from pinecone import PodSpec

from canopy.tokenizer import Tokenizer

from tqdm.auto import tqdm
from canopy.knowledge_base import KnowledgeBase
from canopy.models.data_models import Document
from canopy.knowledge_base import list_canopy_indexes



In [2]:
load_dotenv(find_dotenv())
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
filepath_parquet=os.getenv("FILEPATH_PARQUET")


In [3]:
data_for_upsert=pd.read_parquet(filepath_parquet)
data_for_upsert

,id,text,source
0,bateria-np-f970-decodificada-para-luces-led-y-...,\nBatería NP-F970 de la marca Iluminus. \nCom...,https://contentia.cl/products/bateria-np-f970-...
1,cargador-de-baterias-serie-np-f-multifuncional...,Presentamos el cargador de baterías multifunci...,https://contentia.cl/products/cargador-de-bate...
2,estabilizador-de-smartphone-zhiyun-smooth-q4-c...,\nEl estabilizador de cardán para smartphone ...,https://contentia.cl/products/estabilizador-de...
3,monopodo-de-video-xpro-de-4-secciones-con-base...,El último monopié XPRO de aluminio de 4 seccio...,https://contentia.cl/products/monopodo-de-vide...
4,monopodo-xpro-manfrotto-4-secciones-en-aluminio,El revolucionario monópodp XPRO de 4 secciones...,https://contentia.cl/products/monopodo-xpro-ma...
...,...,...,...
330,ulanzi-mt-11-tripode-flexible-para-camaras-reflex,El trípode Ulanzi MT-11 Octopus admite cámaras...,https://contentia.cl/products/ulanzi-mt-11-tri...
331,ulanzi-l1-pro-lampara-led-todo-terreno-para-ca...,"Luz LED con 4 niveles de potencia regulable, t...",https://contentia.cl/products/ulanzi-l1-pro-la...
332,pages/tienda,"Dirección:Avenida Italia 1392, local 3, Provid...",https://contentia.cl/pages/tienda
333,pages/despachos,Puedes elegir comprar con Retiro en Tienda o c...,https://contentia.cl/pages/despachos


In [4]:
Tokenizer.initialize()
tokenizer = Tokenizer()
spec = PodSpec(environment="gcp-starter")
index_name = "pinecone-docs"

kb = KnowledgeBase(index_name)

if not any(name.endswith(index_name) for name in list_canopy_indexes()):
    kb.create_canopy_index(spec=spec)

kb.connect()

In [5]:
documents = [Document(**row) for _, row in data_for_upsert.iterrows()]

In [6]:
batch_size = 100
for i in tqdm(range(0, len(documents), batch_size)):
    kb.upsert(documents[i: i+batch_size])

  0%|          | 0/4 [00:00<?, ?it/s]